In [ ]:
import os
import numpy as np
import pandas as pd
#examine data file
file_path = ''
data = np.load(file_path, allow_pickle=True).item()
data

In [ ]:
#tcp_data preprocessing

In [89]:
def process_tcp_files(source_dir, target_csv):
    """
    Extracts statistical features from all tcp.npy files in a directory structure and saves to a CSV.
    
    Parameters:
    - source_dir: Path to the folder containing task folders.
    - target_csv: Path to save the resulting CSV file.
    """
    if not os.path.exists(target_csv):
        with open(target_csv, 'w') as f:
            f.write("task_num," + ",".join([f"{stat}_{feature}" for feature in ['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz', 'TCP_qw'] 
                                           for stat in ['mean', 'std', 'min', 'max']]) + "\n")

    all_features = []
    ct = 0
    for root, dirs, files in os.walk(source_dir):
        if "transformed" in root and "tcp.npy" in files:
            ct += 1
            try:
                tcp_path = os.path.join(root, "tcp.npy")
                #print(tcp_path)
                data = np.load(tcp_path, allow_pickle=True).item()

                extracted_data = []

                for sensor_id, readings in data.items():
                    for entry in readings:
                        tcp = entry['tcp']
                        robot_ft = entry['robot_ft']
                        extracted_data.append({
                            "sensor_id": sensor_id,
                            "timestamp": entry['timestamp'],
                            "TCP_x": tcp[0],
                            "TCP_y": tcp[1],
                            "TCP_z": tcp[2],
                            "TCP_qx": tcp[3],
                            "TCP_qy": tcp[4],
                            "TCP_qz": tcp[5],
                            "TCP_qw": tcp[6]
                        })
                


                
                df = pd.DataFrame(extracted_data)

                stat_features = {}
                for i, ft in enumerate(['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz','TCP_qw']):
                    stat_features.update({
                        f"mean_{ft}": df[ft].mean(),
                        f"std_{ft}": df[ft].std(),
                        f"min_{ft}": df[ft].min(),
                        f"max_{ft}": df[ft].max(),
                    })
                #print(stat_features)

                folder_name = root.split(os.sep)[-2]  
                task_label = folder_name 

                row = [task_label] + list(stat_features.values())
                all_features.append(row)

            except Exception as e:
                print(f"Error processing {tcp_path}: {e}")
        # if ct == 3:
        #     break
    header = ["task_num"] + [f"{stat}_{feature}" for feature in ['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz', 'TCP_qw'] 
                             for stat in ['mean', 'std', 'min', 'max']]

    if os.path.exists(target_csv):
        existing_data = pd.read_csv(target_csv)
        new_data = pd.DataFrame(all_features, columns=header)
        final_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        final_data = pd.DataFrame(all_features, columns=header)

    final_data.to_csv(target_csv, index=False)
    print(f"Features saved to {target_csv}")

source_directory = "D:/Filtered_Tasks" 
target_csv_path = r"D:/statistical_features_tcp_dropsampling.csv"
process_tcp_files(source_directory, target_csv_path)


Error processing D:/Filtered_Tasks\task_0001_user_0012_scene_0009_cfg_0001\transformed\tcp.npy: 'NoneType' object is not subscriptable
Error processing D:/Filtered_Tasks\task_0100_user_0015_scene_0006_cfg_0001\transformed\tcp.npy: 'NoneType' object is not subscriptable
Features saved to D:/statistical_features_tcp.csv


C:\Users\78087\AppData\Local\Temp\ipykernel_82844\446210536.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([existing_data, new_data], ignore_index=True)


In [47]:
def process_force_torque_files(source_dir, target_csv):
    """
    Extracts statistical features from all force_torque.npy files and saves them to a CSV.

    Parameters:
    - source_dir: Path to the folder containing task folders.
    - target_csv: Path to save the resulting CSV file.
    """
    # Check if CSV exists, if not create it with a header
    if not os.path.exists(target_csv):
        with open(target_csv, 'w') as f:
            header = ["task_num"] + [f"{stat}_{ft}" for ft in ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz'] 
                                     for stat in ['mean', 'std', 'min', 'max']]
            f.write(",".join(header) + "\n")

    # List to hold rows for the CSV
    all_features = []

    # Walk through all task folders
    ct = 0
    for root, dirs, files in os.walk(source_dir):
        #print(root)
        if "transformed" in root and "force_torque.npy" in files:
            #ct += 1
            try:
                # Load force_torque.npy file
                ft_path = os.path.join(root, "force_torque.npy")
                data = np.load(ft_path, allow_pickle=True).item()

                extracted_data = []
                for key, readings in data.items():
                    for entry in readings:
                        extracted_data.append({
                            "sensor_id": key,
                            "timestamp": entry['timestamp'],
                            "Fx": entry['zeroed'][0],
                            "Fy": entry['zeroed'][1],
                            "Fz": entry['zeroed'][2],
                            "Tx": entry['zeroed'][3],
                            "Ty": entry['zeroed'][4],
                            "Tz": entry['zeroed'][5],
                        })
                df = pd.DataFrame(extracted_data)
                #print(df.head())
                # Extract and concatenate all 'zeroed' force/torque values
                # zeroed_values = []
                # for timestamp, values in data.items():
                #     for entry in values:
                #         # Validate 'zeroed' data
                #         if 'zeroed' in entry and len(entry['zeroed']) == 6:
                #             zeroed_values.append(entry['zeroed'])
                
                # Check for empty data
                # if len(zeroed_values) == 0:
                #     print(f"No valid data in {ft_path}. Skipping.")
                #     continue
                
                # # Convert to NumPy array
                # ft_array = np.array(zeroed_values)
                
                # Compute statistical features
                stat_features = {}
                for i, ft in enumerate(['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']):
                    stat_features.update({
                        f"mean_{ft}": df[ft].mean(),
                        f"std_{ft}": df[ft].std(),
                        f"min_{ft}": df[ft].min(),
                        f"max_{ft}": df[ft].max(),
                    })
                    #print(df[ft].mean(), df[ft].std(), df[ft].min(), df[ft].max())


                # Extract task_num from folder name
                folder_name = root.split(os.sep)[-2]  # Parent folder of 'transformed'
                task_label = folder_name  # Use the full folder name as the label

                # Add task_label as the label instead of task_num
                row = [task_label] + list(stat_features.values())
                #print(f'current row is: {row}')

                # Append row to the list
                all_features.append(row)

            except Exception as e:
                print(f"Error processing {ft_path}: {e}")


    # Save all features to the CSV
    header = ["task_num"] + [f"{stat}_{ft}" for ft in ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz'] 
                             for stat in ['mean', 'std', 'min', 'max']]

    # If file already exists, append rows
    if os.path.exists(target_csv):
        existing_data = pd.read_csv(target_csv)
        new_data = pd.DataFrame(all_features, columns=header)
        final_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        final_data = pd.DataFrame(all_features, columns=header)

    # Save the DataFrame to CSV
    final_data.to_csv(target_csv, index=False)
    print(f"Features saved to {target_csv}")

# Define source and target directories
source_directory = "D:/Filtered_Tasks"  # Update with your path
target_csv_path = "D:/statistical_features_force_torque.csv"

# Run the pipeline
process_force_torque_files(source_directory, target_csv_path)


Error processing D:/Filtered_Tasks\task_0001_user_0012_scene_0009_cfg_0001\transformed\force_torque.npy: 'NoneType' object is not subscriptable
Error processing D:/Filtered_Tasks\task_0100_user_0015_scene_0006_cfg_0001\transformed\force_torque.npy: 'NoneType' object is not subscriptable
Features saved to D:/statistical_features_force_torque.csv


C:\Users\78087\AppData\Local\Temp\ipykernel_82844\843215869.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([existing_data, new_data], ignore_index=True)


In [ ]:
#Down-sampling for reduced model

In [37]:
def process_tcp_files_wdownsampling(source_dir, target_csv, downsample_rate = 3):
    """
    Extracts statistical features from all tcp.npy files in a directory structure and saves to a CSV.
    """
    if not os.path.exists(target_csv):
        with open(target_csv, 'w') as f:
            f.write("task_num," + ",".join([f"{stat}_{feature}" for feature in ['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz', 'TCP_qw'] 
                                           for stat in ['mean', 'std', 'min', 'max']]) + "\n")

    all_features = []
    ct = 0
    for root, dirs, files in os.walk(source_dir):
        if "transformed" in root and "tcp.npy" in files:
            ct += 1
            try:
                tcp_path = os.path.join(root, "tcp.npy")
                #print(tcp_path)
                data = np.load(tcp_path, allow_pickle=True).item()

                extracted_data = []

                for sensor_id, readings in data.items():
                    for entry in readings:
                        tcp = entry['tcp']
                        robot_ft = entry['robot_ft']
                        extracted_data.append({
                            "sensor_id": sensor_id,
                            "timestamp": entry['timestamp'],
                            "TCP_x": tcp[0],
                            "TCP_y": tcp[1],
                            "TCP_z": tcp[2],
                            "TCP_qx": tcp[3],
                            "TCP_qy": tcp[4],
                            "TCP_qz": tcp[5],
                            "TCP_qw": tcp[6]
                        })
                


                
                df = pd.DataFrame(extracted_data)
                #print(f'Raw dataframe before downsampling size: {df.shape}')
                #print(df.head())

                df_downsampled = df.iloc[::downsample_rate, :].reset_index(drop=True)

                if len(df_downsampled) < 3: 
                    print(f"Skipping {tcp_path} due to insufficient data after downsampling.")
                    continue
                    
                #print(f'Down-sampled dataframe before downsampling size: {df_downsampled.shape}')
                #print(df_downsampled.head())

                stat_features = {}
                for i, ft in enumerate(['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz','TCP_qw']):
                    stat_features.update({
                        f"mean_{ft}": df_downsampled[ft].mean(),
                        f"std_{ft}": df_downsampled[ft].std(),
                        f"min_{ft}": df_downsampled[ft].min(),
                        f"max_{ft}": df_downsampled[ft].max(),
                    })

                folder_name = root.split(os.sep)[-2]  
                task_label = folder_name 

                row = [task_label] + list(stat_features.values())

                all_features.append(row)

            except Exception as e:
                print(f"Error processing {tcp_path}: {e}")
        # if ct == 1:
        #     break
    header = ["task_num"] + [f"{stat}_{feature}" for feature in ['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz', 'TCP_qw'] 
                             for stat in ['mean', 'std', 'min', 'max']]

    if os.path.exists(target_csv):
        existing_data = pd.read_csv(target_csv)
        new_data = pd.DataFrame(all_features, columns=header)
        #print(new_data)
        final_data = pd.concat([existing_data, new_data], ignore_index=True)
        #print(final_data)
    else:
        final_data = pd.DataFrame(all_features, columns=header)

    final_data.to_csv(target_csv, index=False)
    print(f"Features saved to {target_csv}")

source_directory = "D:/Filtered_Tasks" 
target_csv_path = r"D:/statistical_features_tcp_dropsampling.csv"

process_tcp_files_wdownsampling(source_directory, target_csv_path)


Error processing D:/Filtered_Tasks\task_0001_user_0012_scene_0009_cfg_0001\transformed\tcp.npy: 'NoneType' object is not subscriptable
Error processing D:/Filtered_Tasks\task_0100_user_0015_scene_0006_cfg_0001\transformed\tcp.npy: 'NoneType' object is not subscriptable
Features saved to D:/statistical_features_tcp_dropsampling.csv


C:\Users\78087\AppData\Local\Temp\ipykernel_165548\3279215482.py:101: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([existing_data, new_data], ignore_index=True)


In [53]:
def process_force_torque_files_wdownsampling(source_dir, target_csv, downsample_rate = 3):
    """
    Extracts statistical features from all force_torque.npy files and saves them to a CSV.
    """
    # Check if CSV exists, if not create it with a header
    if not os.path.exists(target_csv):
        with open(target_csv, 'w') as f:
            header = ["task_num"] + [f"{stat}_{ft}" for ft in ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz'] 
                                     for stat in ['mean', 'std', 'min', 'max']]
            f.write(",".join(header) + "\n")
    all_features = []
    
    ct = 0
    for root, dirs, files in os.walk(source_dir):
        #print(root)
        if "transformed" in root and "force_torque.npy" in files:
            ct += 1
            try:
                ft_path = os.path.join(root, "force_torque.npy")
                data = np.load(ft_path, allow_pickle=True).item()

                extracted_data = []
                for key, readings in data.items():
                    for entry in readings:
                        extracted_data.append({
                            "sensor_id": key,
                            "timestamp": entry['timestamp'],
                            "Fx": entry['zeroed'][0],
                            "Fy": entry['zeroed'][1],
                            "Fz": entry['zeroed'][2],
                            "Tx": entry['zeroed'][3],
                            "Ty": entry['zeroed'][4],
                            "Tz": entry['zeroed'][5],
                        })
                df = pd.DataFrame(extracted_data)

                df_downsampled = df.iloc[::downsample_rate, :].reset_index(drop=True)
                #print(df.head())
                # zeroed_values = []
                # for timestamp, values in data.items():
                #     for entry in values:
                #         if 'zeroed' in entry and len(entry['zeroed']) == 6:
                #             zeroed_values.append(entry['zeroed'])
                
                # if len(zeroed_values) == 0:
                #     print(f"No valid data in {ft_path}. Skipping.")
                #     continue
                
                # ft_array = np.array(zeroed_values)
                
                # Compute statistical features
                stat_features = {}
                for i, ft in enumerate(['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']):
                    stat_features.update({
                        f"mean_{ft}": df_downsampled[ft].mean(),
                        f"std_{ft}": df_downsampled[ft].std(),
                        f"min_{ft}": df_downsampled[ft].min(),
                        f"max_{ft}": df_downsampled[ft].max(),
                    })
                    #print(df[ft].mean(), df[ft].std(), df[ft].min(), df[ft].max())


                folder_name = root.split(os.sep)[-2] 
                task_label = folder_name
                row = [task_label] + list(stat_features.values())
                #print(f'current row is: {row}')

                # Append row to the list
                all_features.append(row)

            except Exception as e:
                print(f"Error processing {ft_path}: {e}")

        # if ct == 1:
        #     break
    header = ["task_num"] + [f"{stat}_{ft}" for ft in ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz'] 
                             for stat in ['mean', 'std', 'min', 'max']]

    if os.path.exists(target_csv):
        existing_data = pd.read_csv(target_csv)
        new_data = pd.DataFrame(all_features, columns=header)
        final_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        final_data = pd.DataFrame(all_features, columns=header)

    # Save the DataFrame to CSV
    final_data.to_csv(target_csv, index=False)
    print(f"Features saved to {target_csv}")

source_directory = "D:/Filtered_Tasks"
target_csv_path = "D:/statistical_features_force_torque_downsampling.csv"

process_force_torque_files_wdownsampling(source_directory, target_csv_path)


Error processing D:/Filtered_Tasks\task_0001_user_0012_scene_0009_cfg_0001\transformed\force_torque.npy: 'NoneType' object is not subscriptable
Error processing D:/Filtered_Tasks\task_0100_user_0015_scene_0006_cfg_0001\transformed\force_torque.npy: 'NoneType' object is not subscriptable
Features saved to D:/statistical_features_force_torque_downsampling.csv


C:\Users\78087\AppData\Local\Temp\ipykernel_165548\4022597207.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([existing_data, new_data], ignore_index=True)


In [21]:
#aggregation

In [41]:
def process_tcp_files_waggreagation(source_dir, target_csv, agg_window = 3):
    """
    Extracts statistical features from all tcp.npy files in a directory structure and saves to a CSV.
    """
    if not os.path.exists(target_csv):
        with open(target_csv, 'w') as f:
            f.write("task_num," + ",".join([f"{stat}_{feature}" for feature in ['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz', 'TCP_qw'] 
                                           for stat in ['mean', 'std', 'min', 'max']]) + "\n")

    all_features = []
    ct = 0
    for root, dirs, files in os.walk(source_dir):
        if "transformed" in root and "tcp.npy" in files:
            ct += 1
            try:
                tcp_path = os.path.join(root, "tcp.npy")
                #print(tcp_path)
                data = np.load(tcp_path, allow_pickle=True).item()

                extracted_data = []

                for sensor_id, readings in data.items():
                    for entry in readings:
                        tcp = entry['tcp']
                        robot_ft = entry['robot_ft']
                        extracted_data.append({
                            "sensor_id": sensor_id,
                            "timestamp": entry['timestamp'],
                            "TCP_x": tcp[0],
                            "TCP_y": tcp[1],
                            "TCP_z": tcp[2],
                            "TCP_qx": tcp[3],
                            "TCP_qy": tcp[4],
                            "TCP_qz": tcp[5],
                            "TCP_qw": tcp[6]
                        })
                


                
                df = pd.DataFrame(extracted_data)
                #print(f'Raw dataframe before downsampling size: {df.shape}')
                #print(df.head(6))

                df_filtered = df[['TCP_x', "TCP_y", "TCP_z", "TCP_qx", "TCP_qy", "TCP_qz", "TCP_qw"]]
                df_aggregated = df_filtered.groupby(df_filtered.index // agg_window).mean()

                if len(df_aggregated) < 3:
                    print(f"Skipping {tcp_path} due to insufficient data after aggregation.")
                    continue
                    
                #print(f'Aggregated dataframe before downsampling size: {df_aggregated.shape}')
                #print(df_aggregated.head())

                stat_features = {}
                for i, ft in enumerate(['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz','TCP_qw']):
                    stat_features.update({
                        f"mean_{ft}": df_aggregated[ft].mean(),
                        f"std_{ft}": df_aggregated[ft].std(),
                        f"min_{ft}": df_aggregated[ft].min(),
                        f"max_{ft}": df_aggregated[ft].max(),
                    })
                #print(stat_features)

                folder_name = root.split(os.sep)[-2] 
                task_label = folder_name

                row = [task_label] + list(stat_features.values())

                all_features.append(row)

            except Exception as e:
                print(f"Error processing {tcp_path}: {e}")
        #if ct == 1:
        #    break
    header = ["task_num"] + [f"{stat}_{feature}" for feature in ['TCP_x', 'TCP_y', 'TCP_z', 'TCP_qx', 'TCP_qy', 'TCP_qz', 'TCP_qw'] 
                             for stat in ['mean', 'std', 'min', 'max']]

    if os.path.exists(target_csv):
        existing_data = pd.read_csv(target_csv)
        new_data = pd.DataFrame(all_features, columns=header)
        #print(new_data)
        final_data = pd.concat([existing_data, new_data], ignore_index=True)
        #print(final_data)
    else:
        final_data = pd.DataFrame(all_features, columns=header)

    final_data.to_csv(target_csv, index=False)
    print(f"Features saved to {target_csv}")

source_directory = "D:/Filtered_Tasks" 
target_csv_path = r"D:/statistical_features_tcp_aggregation.csv"

process_tcp_files_waggreagation(source_directory, target_csv_path)


Error processing D:/Filtered_Tasks\task_0001_user_0012_scene_0009_cfg_0001\transformed\tcp.npy: 'NoneType' object is not subscriptable
Error processing D:/Filtered_Tasks\task_0100_user_0015_scene_0006_cfg_0001\transformed\tcp.npy: 'NoneType' object is not subscriptable
Features saved to D:/statistical_features_tcp_aggregation.csv


C:\Users\78087\AppData\Local\Temp\ipykernel_165548\3184171483.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([existing_data, new_data], ignore_index=True)


In [57]:
def process_force_torque_files_waggregation(source_dir, target_csv, agg_window = 3):
    """
    Extracts statistical features from all force_torque.npy files and saves them to a CSV.
    """
    if not os.path.exists(target_csv):
        with open(target_csv, 'w') as f:
            header = ["task_num"] + [f"{stat}_{ft}" for ft in ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz'] 
                                     for stat in ['mean', 'std', 'min', 'max']]
            f.write(",".join(header) + "\n")

    all_features = []

    ct = 0
    for root, dirs, files in os.walk(source_dir):
        #print(root)
        if "transformed" in root and "force_torque.npy" in files:
            ct += 1
            try:
                ft_path = os.path.join(root, "force_torque.npy")
                data = np.load(ft_path, allow_pickle=True).item()

                extracted_data = []
                for key, readings in data.items():
                    for entry in readings:
                        extracted_data.append({
                            "sensor_id": key,
                            "timestamp": entry['timestamp'],
                            "Fx": entry['zeroed'][0],
                            "Fy": entry['zeroed'][1],
                            "Fz": entry['zeroed'][2],
                            "Tx": entry['zeroed'][3],
                            "Ty": entry['zeroed'][4],
                            "Tz": entry['zeroed'][5],
                        })
                df = pd.DataFrame(extracted_data)

                df_filtered = df[['Fx', "Fy", "Fz", "Tx", "Ty", "Tz"]]
                df_aggregated = df_filtered.groupby(df_filtered.index // agg_window).mean()


                if len(df_aggregated) < 3: 
                    print(f"Skipping {tcp_path} due to insufficient data after aggregation.")
                    continue

                #df_downsampled = df.iloc[::downsample_rate, :].reset_index(drop=True)
                #print(df.head(6))
                #print(df_aggregated.head())
                # zeroed_values = []
                # for timestamp, values in data.items():
                #     for entry in values:
                #         if 'zeroed' in entry and len(entry['zeroed']) == 6:
                #             zeroed_values.append(entry['zeroed'])
                
                # if len(zeroed_values) == 0:
                #     print(f"No valid data in {ft_path}. Skipping.")
                #     continue
                
                # ft_array = np.array(zeroed_values)
                
                stat_features = {}
                for i, ft in enumerate(['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']):
                    stat_features.update({
                        f"mean_{ft}": df_aggregated[ft].mean(),
                        f"std_{ft}": df_aggregated[ft].std(),
                        f"min_{ft}": df_aggregated[ft].min(),
                        f"max_{ft}": df_aggregated[ft].max(),
                    })
                    #print(df[ft].mean(), df[ft].std(), df[ft].min(), df[ft].max())


                folder_name = root.split(os.sep)[-2] 
                task_label = folder_name 

                row = [task_label] + list(stat_features.values())
                #print(f'current row is: {row}')

                all_features.append(row)

            except Exception as e:
                print(f"Error processing {ft_path}: {e}")

        # if ct == 1:
        #     break
    header = ["task_num"] + [f"{stat}_{ft}" for ft in ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz'] 
                             for stat in ['mean', 'std', 'min', 'max']]

    if os.path.exists(target_csv):
        existing_data = pd.read_csv(target_csv)
        new_data = pd.DataFrame(all_features, columns=header)
        final_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        final_data = pd.DataFrame(all_features, columns=header)

    final_data.to_csv(target_csv, index=False)
    print(f"Features saved to {target_csv}")

source_directory = "D:/Filtered_Tasks" 
target_csv_path = "D:/statistical_features_force_torque_aggregation.csv"

# Run the pipeline
process_force_torque_files_waggregation(source_directory, target_csv_path)


Error processing D:/Filtered_Tasks\task_0001_user_0012_scene_0009_cfg_0001\transformed\force_torque.npy: 'NoneType' object is not subscriptable
Error processing D:/Filtered_Tasks\task_0100_user_0015_scene_0006_cfg_0001\transformed\force_torque.npy: 'NoneType' object is not subscriptable
Features saved to D:/statistical_features_force_torque_aggregation.csv


C:\Users\78087\AppData\Local\Temp\ipykernel_165548\2258408637.py:112: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([existing_data, new_data], ignore_index=True)
